In [ ]:
# Install necessary libraries
!pip install paho-mqtt

In [ ]:
# Import necessary libraries
import pandas as pd
import paho.mqtt.client as mqtt
import json
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [ ]:
# TTN application and device details
app_id = "<Username from 'Connection credential' of TTN MQTT Integration>" 
access_key = "<Password from 'Connection credential' of TTN MQTT Integration>" 
device_id = "<End device ID from TTN Application>" 

In [ ]:
# Function to handle connection
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("v3/"+ str(app_id) +"/devices/"+ str(device_id) +"/up")

# Function to handle messages
def on_message(client, userdata, message):
    msg = message.payload.decode('utf-8')
    # Parse the JSON data
    data = json.loads(msg)

    # Extract temperature and humidity fields
    temperature = data['uplink_message']['decoded_payload']['temperature']
    humidity = data['uplink_message']['decoded_payload']['humidity']
    # Create a DataFrame to store temperature and humidity
    temp_hum = {'temperature': temperature, 'humidity': humidity}
    df.loc[len(df)] = temp_hum
    plot_graphs()

# Function to plot temperature and humidity graphs
def plot_graphs():
  if not df.empty:
          # Creating subplots
          fig, ax = plt.subplots(2, 1, figsize=(10, 8))
          # Plot temperature
          ax[0].plot(df.index, df['temperature'], label='Temperature', color='red', marker='o')
          ax[0].set_xlabel('Index')
          ax[0].set_ylabel('Temperature')
          ax[0].set_title('Temperature Plot')
          ax[0].legend()
          # Plot humidity
          ax[1].plot(df.index, df['humidity'], label='Humidity', color='blue', marker='x')
          ax[1].set_xlabel('Index')
          ax[1].set_ylabel('Humidity')
          ax[1].set_title('Humidity Plot')
          ax[1].legend()
          # Adjust layout
          plt.tight_layout()
          # Clear the output to replace it with the new one
          clear_output(wait=True)
          # Show the graphs
          plt.show()

In [ ]:
# Create an empty DataFrame to store device temperature and humidity
columns = ['temperature', 'humidity']
df = pd.DataFrame(columns=columns)

# Connect to TTN MQTT
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION1, 'prova')
client.username_pw_set(username=app_id, password=access_key)
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect("eu1.cloud.thethings.network", 1883, 60)

# Start the MQTT client loop
client.loop_forever()